In [2]:
# Do not delete this cell. It ensures that you can do the imports,
# load datasets etc. in the same fashion as in any Python script
# in the project template.


import sys
sys.path.insert(0, '../..')
from bld.project_paths import project_paths_join as ppj
from bld.project_paths import project_paths as pp



In [3]:
import numpy as np
import json
import pickle
import pandas as pd
from scipy.stats import sem
import seaborn as sns
import matplotlib.pyplot as plt
%config Completer.use_jedi = False

from matplotlib import rc
rc('text', usetex=True)
plt.rcParams.update({'font.size': 20})


# Are fully algortihmic markets more collusive than fully human markets?

To answer this question we will compare the outcomes for the algorithmic market to the outcomes in the last super game of the human markets. We focus on the last super game as this allowed humans for some learning. Arguably this is the most accurate comparision.

First we load the needed data.

In [4]:
# Two firm algorithm markets
with open(ppj("OUT_DATA", f"grid_2_agents.pickle"), "rb") as f:
    all_output_grids_2_agents = pickle.load(f)
all_prices_2_agents_grid = np.array(all_output_grids_2_agents['avg_price'])
with open(ppj("OUT_DATA", "super_star_avg_prices_2_agents.pickle"), "rb") as f:
    super_star_avg_prices_2_agents = pickle.load(f)

# Three firm algorithm markets
with open(ppj("OUT_DATA", f"grid_3_agents.pickle"), "rb") as f:
    all_output_grids_3_agents = pickle.load(f)
all_prices_3_agents_grid = np.array(all_output_grids_3_agents['avg_price'])
with open(ppj("OUT_DATA", "super_star_avg_prices_3_agents.pickle"), "rb") as f:
    super_star_avg_prices_3_agents = pickle.load(f)


In [5]:
# we want to transform those to pandas dataframes as it will be easier to deal with them once we go over to plotting

In [6]:
with open(ppj("OUT_DATA", "data_group_level.pickle"), "rb") as f:
    data_group_level = pickle.load(f)

## Next up we use mann whitney u tests to compare the respective samples.

In [7]:
from scipy.stats import mannwhitneyu, ttest_1samp

### We start by comparing the market prices within a respective market size.

Two firm market:

In [8]:
print("Two firm market. Algo (super star) v human ")
for sg in range(1,4):
    subset_sg = data_group_level.loc[(data_group_level['super_game'] == sg) & 
                                                        (data_group_level['treatment'] == '2H0A')].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()

    p_value = mannwhitneyu(
        subset_sg['winning_price'],
        super_star_avg_prices_2_agents,
        use_continuity=False,
        alternative='two-sided'
    )[-1]
    print(f"Super game: {sg}. P-value: {p_value:.3f}")

Two firm market. Algo (super star) v human 
Super game: 1. P-value: 0.000
Super game: 2. P-value: 0.000
Super game: 3. P-value: 0.000


In [9]:
print("Two firm market. Algo (grid average) v human ")
for sg in range(1,4):
    subset_sg = data_group_level.loc[(data_group_level['super_game'] == sg) & 
                                      (data_group_level['treatment'] == '2H0A')].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()

    p_value = ttest_1samp(
        subset_sg['winning_price'],
        all_prices_2_agents_grid.mean(),
        alternative='two-sided'
    )[-1]
    print(f"Super game: {sg}. P-value: {p_value:.3f}")

Two firm market. Algo (grid average) v human 
Super game: 1. P-value: 0.000
Super game: 2. P-value: 0.005
Super game: 3. P-value: 0.034


In [10]:
print(f"Average price of the grid in two firm markets: {all_prices_2_agents_grid.mean():.3f}")

Average price of the grid in two firm markets: 3.511


Three firm market:


In [11]:
print("Three firm market. Algo (super star) v human ")
for sg in range(1,4):
    subset_sg = data_group_level.loc[(data_group_level['super_game'] == sg) & 
                                     (data_group_level['treatment'] == '3H0A')].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()

    p_value = mannwhitneyu(
        subset_sg['winning_price'],
        super_star_avg_prices_3_agents,
        use_continuity=False,
        alternative='two-sided'
    )[-1]
    print(f"Super game: {sg}. P-value: {p_value:.3f}")

Three firm market. Algo (super star) v human 
Super game: 1. P-value: 0.000
Super game: 2. P-value: 0.008
Super game: 3. P-value: 0.980


In [12]:
print("Two firm market. Algo (grid average) v human ")
for sg in range(1,4):
    subset_sg = data_group_level.loc[(data_group_level['super_game'] == sg) & 
                                      (data_group_level['treatment'] == '3H0A')].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()

    p_value = ttest_1samp(
        subset_sg['winning_price'],
        all_prices_3_agents_grid.mean(),
        alternative='two-sided'
    )[-1]
    print(f"Super game: {sg}. P-value: {p_value:.3f}")

Two firm market. Algo (grid average) v human 
Super game: 1. P-value: 0.991
Super game: 2. P-value: 0.339
Super game: 3. P-value: 0.044


In [13]:
print(f"Average price of the grid in three firm markets: {all_prices_3_agents_grid.mean():.3f}")

Average price of the grid in three firm markets: 1.574


There exist no statistically significant differences between the prices in the three firm markets with the super star algorithms and humans. Interestingly, the average algorithm from the grid search is *less* collusive than the human market with this difference being statistically significant at $p<0.01$.

# Algo v Algo

See Algo analysis q_env.ipynb


# Human v humans


In [14]:
data_humans_first_super_game = data_group_level.loc[(data_group_level['super_game'] == 1) & 
                                                    (data_group_level['treatment'].isin(['3H0A', '2H0A']))].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()
data_humans_second_super_game = data_group_level.loc[(data_group_level['super_game'] == 2) & 
                                                    (data_group_level['treatment'].isin(['3H0A', '2H0A']))].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()
data_humans_third_super_game = data_group_level.loc[(data_group_level['super_game'] == 3) & 
                                                    (data_group_level['treatment'].isin(['3H0A', '2H0A']))].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()


In [15]:
data_humans_all_super_game = data_group_level.loc[
                                                    (data_group_level['treatment'].isin(['3H0A', '2H0A']))].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()


In [16]:
mannwhitneyu(
    data_humans_first_super_game.loc[data_humans_first_super_game['treatment'] == '3H0A']['winning_price'],
    data_humans_first_super_game.loc[data_humans_first_super_game['treatment'] == '2H0A']['winning_price'],
    use_continuity=False,
    alternative='two-sided'

)[-1]

0.04530386472278079

In [17]:
mannwhitneyu(
    data_humans_second_super_game.loc[data_humans_second_super_game['treatment'] == '3H0A']['winning_price'],
    data_humans_second_super_game.loc[data_humans_second_super_game['treatment'] == '2H0A']['winning_price'],
    use_continuity=False,
    alternative='two-sided'

)[-1]

0.050961936967763306

In [18]:
mannwhitneyu(
    data_humans_third_super_game.loc[data_humans_third_super_game['treatment'] == '3H0A']['winning_price'],
    data_humans_third_super_game.loc[data_humans_third_super_game['treatment'] == '2H0A']['winning_price'],
    use_continuity=False,
    alternative='two-sided'

)[-1]

0.28275614744399247

In [19]:
mannwhitneyu(
    data_humans_all_super_game.loc[data_humans_all_super_game['treatment'] == '3H0A']['winning_price'],
    data_humans_all_super_game.loc[data_humans_all_super_game['treatment'] == '2H0A']['winning_price'],
    use_continuity=False,
    alternative='two-sided'

)[-1]

0.04042397933690852

This shows the importance of learning for humans, which appears to be more pronounce in larger and therefore strategically more complex markets.

# Mixed market analysis

#### Different states

There are states that trigger responses by the algorithm that are different from the ones discussed in the paper. Below you see that those can account for only 1% of all rounds.

In [20]:
data_algo = data_group_level.dropna(subset=['price_algorithm']).copy()

In [21]:
len(data_algo.loc[data_algo['price_algorithm'] == 2]) +  len(data_algo.loc[data_algo['price_algorithm'] == 3]) 

49

In [22]:
49 / len(data_algo)

0.012493625701172871

#### Analysis

In [23]:
data_mixed_pooled = data_group_level.loc[(data_group_level['treatment'].isin(['3H0A', '2H0A', '1H1A', '2H1A', '1H2A'])
                                                    )].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()


2H0A v 1H1A (pooled)

In [24]:
mannwhitneyu(
data_mixed_pooled.loc[data_mixed_pooled['treatment'] == '2H0A']['winning_price'],
data_mixed_pooled.loc[data_mixed_pooled['treatment'] == '1H1A']['winning_price'],
use_continuity=False,
alternative='two-sided'

)[-1]


0.83563115352624

3H0A v 2H1A

In [25]:
mannwhitneyu(
data_mixed_pooled.loc[data_mixed_pooled['treatment'] == '3H0A']['winning_price'],
data_mixed_pooled.loc[data_mixed_pooled['treatment'] == '2H1A']['winning_price'],
use_continuity=False,
alternative='two-sided'

)[-1]


0.06391934147515746

2H1A v 1H2A

In [26]:
mannwhitneyu(
data_mixed_pooled.loc[data_mixed_pooled['treatment'] == '2H1A']['winning_price'],
data_mixed_pooled.loc[data_mixed_pooled['treatment'] == '1H2A']['winning_price'],
use_continuity=False,
alternative='two-sided'

)[-1]


0.005276795014053497

3H0A v 1H2A

In [27]:
mannwhitneyu(
data_mixed_pooled.loc[data_mixed_pooled['treatment'] == '3H0A']['winning_price'],
data_mixed_pooled.loc[data_mixed_pooled['treatment'] == '1H2A']['winning_price'],
use_continuity=False,
alternative='two-sided'

)[-1]


0.17542896105941352

,super_group_id_general,treatment,winning_price,collusive
0,session_02062021_1000_1,3H0A,1.327044,0.037736
1,session_02062021_1000_2,3H0A,2.207547,0.364780
2,session_02062021_1130_1,3H0A,1.238994,0.075472
3,session_02062021_1300_1,2H1A,1.069182,0.000000
4,session_02062021_1300_2,2H1A,1.559748,0.176101
...,...,...,...,...
72,session_25052021_1200_6,1H1A,4.000000,1.000000
73,session_25052021_1200_7,1H1A,2.886792,0.528302
74,session_25052021_1200_8,1H1A,1.528302,0.113208
75,session_25052021_1400_1,2H0A,2.704403,0.679245


# Weird states

In [28]:
with open(ppj("OUT_DATA", "data_individual_level.pickle"), "rb") as f:
    data_individual_level = pickle.load(f)

In [54]:
# Share of states where an algo play smth else than 1 or 4
data_algos_group = data_group_level.loc[~data_group_level['treatment'].isin(['2H0A', '3H0A'])].copy()
n_price_not_1_or_4 = data_algos_group.loc[~data_algos_group['price_algorithm'].isin([1,4])] 


0.012493625701172871

In [55]:
data_algos = data_individual_level.loc[~data_individual_level['treatment'].isin(['2H0A', '3H0A'])].copy()

In [64]:
data_algos.loc[(data_algos['price_algorithm'] == 4) & (~data_algos['price_lag_1'].isin([1,4])) & 
               (data_algos['round']>1)]

,participant.code,round,super_game,group_id,winning_price,price,price_algorithm,data_identifier,treatment,super_group_id,...,winning_price_lag_2,price_lag_2,price_algorithm_2,winning_price_lag_3,price_lag_3,price_algorithm_3,winning_price_lag_4,price_lag_4,price_algorithm_4,r_sg
322,4ekr2gwa,2.0,2.0,11,4.0,4.0,4.0,session_20052021_1200,1H1A,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26
617,dsqoki32,10.0,3.0,6,4.0,4.0,4.0,session_21052021_1200,1H1A,6,...,4.0,4.0,4.0,4.0,4.0,4.0,1.0,1.0,1.0,51


In [67]:
(len(n_price_not_1_or_4) + 2 )/ len(data_algos_group)

0.013003569607343193